In [2]:
## Imports
import os
import numpy as np                    
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import FuncAnimation
import seaborn as sns
from skimage import io as sio
from Lab_Analyses.Spine_Analysis_v2 import spine_utilities as s_utils
from Lab_Analyses.Utilities.save_load_pickle import load_pickle
from Lab_Analyses.Video_Processing.smooth_trim_video import smooth_trim_video

sns.set()
sns.set_style("ticks")

In [67]:
# Load in output from GUI
mouse_id = "JW306"
file_name = r"G:\Analyzed_data\individual\JW306\imaging\FOV2\Early\JW306_240326_GreenCh_imaging_data.pickle"
data = load_pickle([file_name])[0]

file_name2 = r"G:\Analyzed_data\individual\JW306\imaging\FOV2\Early\JW306_240326_RedCh_imaging_data.pickle"
data2 = load_pickle([file_name2])[0]

In [ ]:
# Get activity related data
spine_activity = data.activity_trace["Spine"]
spine_dFoF = data.processed_dFoF["Spine"]
dendrite_activity = data2.activity_trace["Dendrite"]
dendrite_dFoF = data2.processed_dFoF["Dendrite"]
#spine_calcium = data.spine_calcium_processed_dFoF

In [119]:
dendrite_activity

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [120]:
# Mask active and inactive portions
def get_active_inactive(dFoF, active):
    inactive_mask = (active == 1)
    active_mask = (active == 0)
    active_dFoF = np.copy(dFoF)
    inactive_dFoF = np.copy(dFoF)
    active_dFoF[active_mask] = np.nan
    #inactive_dFoF[inactive_mask] = np.nan
    
    return active_dFoF, inactive_dFoF

In [121]:
spine_active = np.zeros(spine_activity.shape) * np.nan
spine_inactive = np.zeros(spine_activity.shape) * np.nan
#dend_active = np.zeros(dendrite_activity.shape) * np.nan
#dend_inactive = np.zeros(dendrite_activity.shape) * np.nan

for i in range(spine_activity.shape[1]):
    s_active, s_inactive = get_active_inactive(spine_dFoF[:, i], spine_activity[:, i])
    spine_active[:, i] = s_active
    spine_inactive[:, i] = s_inactive

dend_active, dend_inactive = get_active_inactive(dendrite_dFoF.flatten(), dendrite_activity.flatten())
dend_active[38700:38800] = np.nan

In [154]:
spine_active[40080:40100] = np.nan

In [157]:
# Select the spine idx to look at
spine_idxs = [39, 48]

spine_idxs = [30]

In [158]:
# Visualize the activity
%matplotlib qt

fig, ax = plt.subplot_mosaic("""A""",figsize=(7,3))
title = f"{mouse_id}_FOV2_spine_local_coactivity"
fig.suptitle(title)

frames = np.arange(len(spine_active))
for i, spine in enumerate(spine_idxs):
    ax["A"].plot(frames[:], spine_inactive[:, spine] + i, color="blue", alpha=0.3, linewidth=.8)
    ax["A"].plot(frames[:], spine_active[:, spine] + i, color="blue", linewidth=.8)
    
ax["A"].plot(frames[:], dend_inactive[:] + 2, color="red", alpha=0.3, linewidth=.8)
ax["A"].plot(frames[:], dend_active[:] + 2, color="red", linewidth=.8)

In [159]:
# Specify the frame range to process the video
frame_range = (36500, 41000)
# Replot
fig, ax = plt.subplot_mosaic("""A""",figsize=(7,3))
title = f"{mouse_id}_FOV2_spine_local_coactivity"
fig.suptitle(title)
frames = np.arange(len(spine_active))
for i, spine in enumerate(spine_idxs):
    ax["A"].plot(frames[frame_range[0]:frame_range[1]], (spine_inactive[frame_range[0]:frame_range[1], spine]) / 2 + i, color="blue", alpha=0.3, linewidth=.8)
    ax["A"].plot(frames[frame_range[0]:frame_range[1]], (spine_active[frame_range[0]:frame_range[1], spine]) / 2 + i, color="blue", linewidth=.8)
    
ax["A"].plot(frames[frame_range[0]:frame_range[1]], dend_inactive[frame_range[0]:frame_range[1]] + 1, color="red", alpha=0.3, linewidth=.8)
ax["A"].plot(frames[frame_range[0]:frame_range[1]], dend_active[frame_range[0]:frame_range[1]] + 1, color="red", linewidth=.8)

In [92]:
# Set up video paths
image_dir = r"Z:\People\Jake\Imaging_Data\JW306\240326\FOV2\GreenCh"
out_name = f"JW306_FOV2_frame_{frame_range[0]}_{frame_range[1]}_GreenCh"
save_dir = r"C:\Users\Jake\Desktop\Figures\Examples\Example_videos"
smooth_window = 30 # In frames

In [93]:
# Smooth the images to match processed dFoF and trim to only frames needed
frame_range_vid = (frame_range[0]+200, frame_range[1]+200)
smooth_trim_video(
    image_dir=image_dir,
    out_name=out_name,
    save_dir=save_dir,
    frame_range=frame_range_vid,
    smooth_window=smooth_window,
)

Frames 36000 - 41600
Start 36700  End 41200
File List:
45 : JW306_0326_00003_00002_corrected.tif
46 : JW306_0326_00003_00003_corrected.tif
47 : JW306_0326_00003_00004_corrected.tif
48 : JW306_0326_00003_00005_corrected.tif
49 : JW306_0326_00003_00006_corrected.tif
50 : JW306_0326_00003_00007_corrected.tif
51 : JW306_0326_00003_00008_corrected.tif
(100, 512, 512)
(800, 512, 512)
(800, 512, 512)
(800, 512, 512)
(800, 512, 512)
(800, 512, 512)
(400, 512, 512)
Done Processing


In [95]:
from Lab_Analyses.Utilities.activity_timestamps import get_activity_timestamps
def get_coactive_chunks(coactivity):
    chunks = get_activity_timestamps(coactivity)
    return chunks

In [101]:
print(dendrite_activity.flatten()[frame_range[0]:frame_range[1]])

[0. 0. 0. ... 0. 0. 0.]


In [108]:
# Denote coactive periods
spine_1_activity = spine_activity[frame_range[0]:frame_range[1], 30]
#spine_3_activity = spine_activity[frame_range[0]:frame_range[1], 48]
dend_activity = dendrite_activity.flatten()[frame_range[0]:frame_range[1]]

## Spine 1 with Spine 2 coactivity


spine_1_3_coactivity = spine_1_activity * dend_activity
coactivity_chunks = get_coactive_chunks(spine_1_3_coactivity)

spine_activity_chunks = get_coactive_chunks(spine_1_activity)
dend_activity_chunks = get_coactive_chunks(dend_activity)

s_indexes = [(x[0]+22, x[1]+22) for x in spine_activity_chunks]

d_indexes = [(x[0]+22, x[1]+22) for x in dend_activity_chunks]

c_indexes = [(x[0]+22, x[1]+22) for x in coactivity_chunks]


print(s_indexes)
print("")
print("")
print(d_indexes)
print("")
print("")
print(c_indexes)

[(295, 347), (363, 420), (648, 685), (781, 809), (939, 970), (1057, 1094), (1227, 1260), (1521, 1560), (2078, 2176), (2728, 2778), (2795, 2829), (2954, 2998), (3108, 3145), (3605, 3619), (3668, 3695), (4302, 4501)]


[(298, 455), (2239, 2283), (3156, 3337), (3618, 3691), (4008, 4067)]


[(298, 347), (363, 420), (3618, 3619), (3668, 3691)]


In [160]:
coactivity_chunks.pop(2)

(3596, 3597)

In [41]:
remove_idxs = [2,7,9,11]
print(spine_1_3_chunks)
spine_1_3_chunks = [x for i, x in enumerate(spine_1_3_chunks) if i not in remove_idxs]

print(spine_1_3_chunks)

[(44, 93), (278, 283), (313, 327), (407, 419), (453, 479), (1056, 1078), (1178, 1264), (3453, 3471), (4480, 4528), (4616, 4627), (4696, 4729), (4780, 4783), (4938, 4959)]
[(44, 93), (278, 283), (407, 419), (453, 479), (1056, 1078), (1178, 1264), (4480, 4528), (4696, 4729), (4938, 4959)]


In [128]:
# Specify image path to load in after further processing in imageJ
g_image = r"C:\Users\Jake\Desktop\Figures\Examples\Example_videos\JW306_FOV2_frame_36500_41000_GreenCh_cropped_outline_labeld_rgb.tif"
green_image = sio.imread(g_image, plugin="tifffile")

r_image = r"C:\Users\Jake\Desktop\Figures\Examples\Example_videos\JW306_FOV2_frame_36500_41000_RedCh_cropped_outline_labeld_rgb.tif"
red_image = sio.imread(r_image, plugin="tifffile")

m_image = r"C:\Users\Jake\Desktop\Figures\Examples\Example_videos\JW306_FOV2_frame_36500_41000_Merged_cropped_outline_labeled_rgb.tif"
merged_image = sio.imread(m_image, plugin="tifffile")

In [162]:
# Animate plots
spine_idx = 30
frame_range_plot = (frame_range[0]-20, frame_range[1]-20)
## Function to update line plots
def update_lines(
    num, 
    active1, 
    active_line1, 
    inactive1, 
    inactive_line1, 
    active3, 
    active_line3, 
    inactive3, 
    inactive_line3,
    green_images, 
    spine_image1,
    red_images,
    spine_image2,
    merged_images,
    spine_image3
):
    active_line1.set_data(x[:num], active1[:num])
    inactive_line1.set_data(x[:num], inactive1[:num])
    active_line3.set_data(x[:num], active3[:num])
    inactive_line3.set_data(x[:num], inactive3[:num])
    spine_image1.set_array(green_images[num, :, :, :])
    spine_image2.set_array(red_images[num, :, :, :])
    spine_image3.set_array(merged_images[num, :, :, :])
    
    return active_line1, inactive_line1,active_line3, inactive_line3, spine_image1, spine_image2, spine_image3,


plot_inactivity_1 = spine_inactive[frame_range_plot[0]:frame_range_plot[1], spine_idx] / 2
plot_activity_1 = spine_active[frame_range_plot[0]:frame_range_plot[1], spine_idx] / 2
plot_inactivity_3 = (dend_inactive[frame_range_plot[0]:frame_range_plot[1]]) +1
plot_activity_3 = (dend_active[frame_range_plot[0]:frame_range_plot[1]]) + 1
x = np.arange(len(plot_activity_1))

def init():
    spine_image1.set_data(green_image[0, :, :, :])
    spine_image2.set_data(red_image[0, :, :, :])
    spine_image3.set_data(merged_image[0, :, :, :])
    active_line1.set_data([], [])
    inactive_line1.set_data([], [])
    active_line3.set_data([], [])
    inactive_line3.set_data([], [])
    
    return active_line1, inactive_line1, active_line3, inactive_line3, spine_image1, spine_image2, spine_image3


fig, axes = plt.subplot_mosaic(
    """
    ABC
    DDD
    """,
    height_ratios=[2,1],
    figsize=(12,8),
)

# Initialize line plots
active_line1, = axes["D"].plot([], [], color="blue", alpha=1, linewidth=.8, label="active")
inactive_line1, = axes["D"].plot([], [], color="blue", alpha=0.3, linewidth=.8, label="inactive")
active_line3, = axes["D"].plot([], [], color="crimson", alpha=1, linewidth=.8, label="active")
inactive_line3, = axes["D"].plot([], [], color="crimson", alpha=0.3, linewidth=.8, label="inactive")

# Initizalie the image
spine_image1 = axes["A"].imshow(green_image[0, :, :, :])
spine_image2 = axes["B"].imshow(red_image[0, :, :, :])
spine_image3 = axes["C"].imshow(merged_image[0, :, :, :])

# Add coactive periods

    
for chunk in coactivity_chunks:
    chunk_1_3 = axes["D"].axvspan(chunk[0]+20, chunk[1]+20, alpha=0.2, color="magenta")



axes["D"].legend(
    [active_line1,active_line3,chunk_1_3,], 
    ["Spine",  "Dendrite", "Spine-dendrite coactive",], 
    loc=9, ncols=6, framealpha=1
)
axes["D"].set_xlabel("Frames")
axes["D"].set_ylabel("dF/F")
axes["D"].set_xlim(0, x[-1])
axes["D"].set_ylim(-0.1, 2)
axes["A"].axis("off")
axes["B"].axis("off")
axes["C"].axis("off")

axes["A"].set_title("iGluSnFR3")
axes["B"].set_title("RCaMP2")
axes["C"].set_title("Merged (spine masked)")

fig.tight_layout()

# Setup the Writer for saving
Writer = animation.FFMpegWriter(fps=120)


line_animation = animation.FuncAnimation(
    fig, update_lines, init_func=init, frames=len(plot_inactivity_1), 
    fargs=(
        plot_activity_1, 
        active_line1, 
        plot_inactivity_1, 
        inactive_line1,
        plot_activity_3, 
        active_line3, 
        plot_inactivity_3, 
        inactive_line3,
        green_image, 
        spine_image1,
        red_image,
        spine_image2,
        merged_image,
        spine_image3,), 
    interval=0.5, blit=True, repeat=True
)

animation_fname = r"C:\Users\Jake\Desktop\Figures\Examples\Example_videos\Coincident_Activity_Video.mp4"

line_animation.save(animation_fname, writer=Writer)

In [44]:
print(plt.rcParams["animation.bitrate"])

-1


In [ ]:
image.shape

In [ ]:
plt.imshow(np.ones((200,211,3)))

In [ ]:
a = np.array([2,1,1,2,4,5,1,7,])
np.unique(a)